In [15]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from peft import LoraConfig, get_peft_model
import torch

In [16]:
# 1) Load your chunk data
df_chunks = pd.read_csv("./output_org/final_chunk_df.csv")
df_chunks

,text,video,start,end,Chunk_hate
0,The End Thank you. The End,hate_video_1,29.98,94.96,0
1,I Like Sugar And the NAACP would sure like to...,hate_video_2,5.22,25.44,1
2,Roses are red and violets are blue and nigger...,hate_video_2,27.42,32.12,1
3,"But they don't mind cause what the heck, you ...",hate_video_2,32.42,37.80,1
4,"No joke, I ain't got a nickel for a coke and ...",hate_video_2,38.34,44.08,1
...,...,...,...,...,...
32437,public opinion on the internet. Israel's fore...,non_hate_video_652,29.40,34.24,0
32438,"so-called trainee diplomats, to skew online p...",non_hate_video_652,34.36,39.92,0
32439,"line. This is an organized effort using, quot...",non_hate_video_652,39.92,55.12,0
32440,They already were able to remove my If Americ...,non_hate_video_652,55.24,59.82,0


In [17]:
# -----------------------------
# 1) Prepare your labeled training examples
# -----------------------------
train_data = {
    "text": [
    "I fucking hate niggers.",
    "Oh boy, what the fuck are you doing?",
    "Jews are always there when you do not want",
    "Hey I told you to get rid of that shit",
    "Muslim people are filthy and should all be killed",
    "Sorry I don't understand",
    "I fucking hate nigga, gorilla, monkey boy, you black motherfucker.",
    "Get the hell out of my face",
    "Dirty coon go back to where you belong",
    "Sorry for the other day, but I am ready to fight if you are or are not",
    "Fuck Jews, they all must die",
    "I don't give two shits about your friend"
],
    "label": [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]  # 1 = HATE, 0 = NON_HATE
}
train_df = pd.DataFrame(train_data)
raw_train = Dataset.from_pandas(train_df.reset_index(drop=True))
raw_train = raw_train.rename_column("label", "labels")

# -----------------------------
# 2) Load model, tokenizer, and attach LoRA adapter
# -----------------------------
model_name = "Hate-speech-CNERG/dehatebert-mono-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["classifier"],
    lora_dropout=0.1,
    bias="none"
)
model = get_peft_model(base_model, lora_config)

# -----------------------------
# 3) Tokenization function
# -----------------------------
max_len = 128
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=max_len)

# -----------------------------
# 4) Prepare training dataset and data collator
# -----------------------------
train_ds = raw_train.map(tokenize, batched=True)
train_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
collator = DataCollatorWithPadding(tokenizer)

# -----------------------------
# 5) Training configuration with explicit max steps
# -----------------------------
t_args = TrainingArguments(
    output_dir="fewshot_dehatebert",
    per_device_train_batch_size=4,
    # Stop training after 25 update steps
    max_steps=25,
    logging_steps=5,
    save_steps=5,
    fp16=True,
    remove_unused_columns=False
)

t_args = TrainingArguments(
    output_dir="fewshot_dehatebert",
    per_device_train_batch_size=4,
    num_train_epochs=10,
    logging_steps=5,
    save_steps=10,
    fp16=True,
    remove_unused_columns=False
)

# -----------------------------
# 6) Custom Trainer to ignore extra kwargs
# -----------------------------
class FilteringTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels", None)
        inputs = {k: v for k, v in inputs.items() if k in ["input_ids", "attention_mask"]}
        if labels is not None:
            inputs["labels"] = labels
        outputs = model(**inputs)
        loss = outputs.loss if hasattr(outputs, 'loss') else outputs[0]
        return (loss, outputs) if return_outputs else loss

trainer = FilteringTrainer(
    model=model,
    args=t_args,
    train_dataset=train_ds,
    data_collator=collator,
)

# -----------------------------
# 7) Train the LoRA adapter
# -----------------------------
trainer.train()

# -----------------------------
# 8) Inference on your updated CSV file
# -----------------------------
# Replace 'chunks_with_hate_labels.csv' with your actual CSV path containing a 'text' column
raw_chunks = Dataset.from_pandas(df_chunks.reset_index(drop=True))

# Tokenize chunks
infer_ds = raw_chunks.map(tokenize, batched=True)
infer_ds.set_format(type="torch", columns=["input_ids", "attention_mask"])

# Predict
predictions = trainer.predict(infer_ds)
logits = predictions.predictions
pred_ids = torch.argmax(torch.tensor(logits), axis=1).tolist()

# Map and add to DataFrame
label_map = {0: "NON_HATE", 1: "HATE"}
df_chunks["predicted_label"] = [label_map[i] for i in pred_ids]

# Print the DataFrame with new labels
df_chunks.to_excel("final_results.xlsx")


Map: 100%|██████████| 12/12 [00:00<?, ? examples/s]
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
5,0.551600
10,0.302300
15,0.541600
20,0.537900
25,0.260500
30,0.418200


Map: 100%|██████████| 32442/32442 [00:01<00:00, 18694.11 examples/s]


In [18]:
df_chunks["predicted_hate"] = df_chunks["predicted_label"].apply(lambda val: 1 if val == "HATE" else 0)

new_df_chunks = df_chunks.copy()

new_df_chunks.loc[
    (new_df_chunks["Chunk_hate"] == 0) & (new_df_chunks["predicted_label"] == "HATE"),
    "Chunk_hate"
] = 1


assert len(new_df_chunks[(new_df_chunks["Chunk_hate"] == 0) & (new_df_chunks["predicted_hate"] == 1)]) == 0

In [19]:
new_df_chunks["Chunk_hate"].value_counts()

Chunk_hate
0    23516
1     8926
Name: count, dtype: int64

In [20]:
new_df_chunks.drop(['predicted_label','predicted_hate'], axis = 1).to_excel("./output_org/final_res.xlsx")

In [21]:
new_df_chunks

,text,video,start,end,Chunk_hate,predicted_label,predicted_hate
0,The End Thank you. The End,hate_video_1,29.98,94.96,0,NON_HATE,0
1,I Like Sugar And the NAACP would sure like to...,hate_video_2,5.22,25.44,1,HATE,1
2,Roses are red and violets are blue and nigger...,hate_video_2,27.42,32.12,1,HATE,1
3,"But they don't mind cause what the heck, you ...",hate_video_2,32.42,37.80,1,NON_HATE,0
4,"No joke, I ain't got a nickel for a coke and ...",hate_video_2,38.34,44.08,1,NON_HATE,0
...,...,...,...,...,...,...,...
32437,public opinion on the internet. Israel's fore...,non_hate_video_652,29.40,34.24,0,NON_HATE,0
32438,"so-called trainee diplomats, to skew online p...",non_hate_video_652,34.36,39.92,0,NON_HATE,0
32439,"line. This is an organized effort using, quot...",non_hate_video_652,39.92,55.12,0,NON_HATE,0
32440,They already were able to remove my If Americ...,non_hate_video_652,55.24,59.82,0,NON_HATE,0
